# Overview

In this part, we take our initial transformed data;

- HPIInitialTransform.csv
- HPINational.csv
- MedianPayInitialTransform.csv
- MedianSalaryNational.csv
- CPIHInitialTransform.csv

and load it into MySQL Workbench, to clean and model it.

Recall that in the initial cleaning phase, we did not check for missing values, or inconsistent/inappropriate data types (e.g., text used for dates), or verify that related tables represent data in a consistent format (which will be required for facilitating joins).

The aim of this next phase is to create a schema resembling the following: (note: dates within the housing tables will be foreign keys, as will all mentions of regions. On the other hand, the date-related fields in CPI and median salary tables will form a many-to-many relationship with the dates dimension table — creating separate dimension tables for years and months felt like overkill).

![Schema showing the tables, columns and relationships between them](documentation_images\ModellingDiagram.png)

A remark on why both regional and national versions of the Housing Prices and Median Salary data exist:

- Housing Prices: The sales volumes aren't available for the different types of houses sold (detached, semi-detached, terraced, and flats), so we cannot reconstruct the national average sale price by type without making assumptions about the distribution of each type sold, (and so we must include national level averages for each type).

    The `average_price` and `sales_volume` are also included since the national figures for these are available, and we may as well use them (it'll save us doing a calculation later).
  
- Median Salaries: The median of regional median salaries does not equal the national median salary (this is not due to a data issue, but because of the nature of how medians are calculated). As a result, we require both regional and national median salary statistics for accurate analysis at each level.

# Loading the data

We begin by loading the data from the previous section (after its initial clean in Excel/PowerQuery). We load the data into MySQL Workbench using the Table Data Import Wizard.

![A snapshot of the data loaded into MySQL Workbench](documentation_images\ModellingSQLLoadIn.png)

# Creating the dimension tables

We start by creating the dimension tables. The Regions table will consist of a single column containing the 12 NUTS regions of the UK: North East, North West, Yorkshire and the Humber, East Midlands, West Midlands, East of England, Greater London, South East, South West, Wales, Scotland, and Northern Ireland.

Since we want this table to serve as the source for the correct formatting and spelling of the region names, we will manually create it rather than derive it through the filtering of other tables.

This is done as follows:

```sql
CREATE TABLE RegionsDim (
    region varchar(30) primary key
);

INSERT INTO RegionsDim (region)
VALUES
("North East"),
("North West"), 
("Yorkshire and the Humber"), 
("East Midlands"), 
("West Midlands"),
("East of England"),
("London"), 
("South East"), 
("South West"), 
("Wales"), 
("Scotland"), 
("Northern Ireland");
```

Viewing this table to verify:

```sql
select * from RegionsDim;
```

![The output of the query `select * from RegionsDim;`](documentation_images\RegionsDim.png)

To build an efficient dates dimension table, we want to find a reasonable minimum and maximum date based on the dates in the fact tables.

To do this, we query for the minimum date-related value from each of our five tables (there will be duplicates, but better to be safe than sorry), and visually compare the date-related values. We do the same for the maximum date-related values. Finally, we create the `DatesDim` table based on the identified date range.


```sql
--to find the minimum
select *
from (SELECT MonthYear as d1 FROM cpihinitialtransform LIMIT 1) as t1
cross join (SELECT hpiinitialtransform.Date as d2 FROM hpiinitialtransform LIMIT 1) as t2
cross join (SELECT hpinational.Date as d3 FROM hpinational LIMIT 1) as t3
cross join (SELECT medianpayinitialtransform.Year as d4 FROM medianpayinitialtransform LIMIT 1) as t4
cross join (SELECT mediansalarynational.Year as d5 FROM mediansalarynational LIMIT 1) as t5;
```
Output:
![output of querying for the minimum date-related value from each table](documentation_images\minimum_dates.png)


```sql
--to find the maximum
select *
from (SELECT MonthYear as d1 FROM cpihinitialtransform ORDER BY MonthYear DESC LIMIT 1) as t1
cross join (SELECT hpiinitialtransform.Date as d2 FROM hpiinitialtransform ORDER BY hpiinitialtransform.Date DESC LIMIT 1) as t2
cross join (SELECT hpinational.Date as d3 FROM hpinational ORDER BY hpinational.Date DESC LIMIT 1) as t3
cross join (SELECT medianpayinitialtransform.Year as d4 FROM medianpayinitialtransform ORDER BY medianpayinitialtransform.Year DESC LIMIT 1) as t4
cross join (SELECT mediansalarynational.Year as d5 FROM mediansalarynational ORDER BY mediansalarynational.Year DESC LIMIT 1) as t5;
```
Output:
![output of querying for the maximum date-related value from each table](documentation_images\maximum_dates.png)

So, CPI starts in Jan 1988 and ends in Jan 2025, House Prices start in April 1968 and end in Dec 2024, and median pay figures (annual) start in 1999 and end in 2023.

Therefore, our minimum date will be `01/04/1968` and our maximum date will be `1/12/2024` (we will discard the first house prices reading of Jan 2025).

We now create the date dimension table, `DatesDim`. To do this, we use a `INSERT INTO SELECT` query alongside a recursive CTE to generate a range of dates from `01/04/1988` to `1/12/2024` with monthly intervals. Then use the `MONTH` and `YEAR` functions to extract the month (as a value from 1-12) and the year from each date. 

```sql
create table DatesDim (
    date DATE primary key,
    month INT,
    year INT
);

insert into DatesDim (date, month, year)
	-- CTE with range of dates from April 1988 to Dec 2024 in yyyy-mm-dd format
	with recursive dates as (
		select CAST('1968-04-01' as date) as d
		union
		select DATE_ADD(d, interval 1 month)
		from dates
		where d < '2024-12-01'
	)
select d, MONTH(d) as month, YEAR(d) as year
from dates;
```

Printing the head of `DatesDim` to verify the rows are as expected:

```sql 
select * from DatesDim limit 5;
```

Output:
![The first five rows of the DatesDim table](documentation_images\DatesDimHead.png)

# Facts tables - cleaning, and relationships to the dimension tables

For the final stage of the modelling and cleaning process, we clean the five fact tables and establish relationships with the dimension tables. Since the cleaning steps for the regional and national tables will be largely similar, we will walk through the process only for the regional tables.

## House Price tables (regional walkthrough, national is essentially identical)

We first query for a description of the regional house price data (`hpiinitialtransform`) to see what datatypes each of the columns have. We also query for a random sample of 10 rows of the data, to see what we are working with (done using `ORDER BY RAND()`):

```sql
desc hpiinitialtransform;
```

```sql
select * from hpiinitialtransform order by rand();
```

Outputs:

![description of columns of housing price data](documentation_images\DescriptionHPI.png)

![sample of 10 random rows of housing price data](documentation_images\Sample10HPI.png)


Since some column datatypes need to be changed, we handle the cleaning process by creating a new table, HousePricesRegional, which includes the converted columns. We also define foreign key constraints linking to the DatesDim and RegionsDim dimension tables to ensure integrity of the data.

To populate the new table, we use `CREATE TABLE ... AS SELECT`, casting and transforming columns from the `hpiinitialtransform` table. For numeric columns, blank values are replaced with `NULL` rather than `0`, to avoid distorting aggregate functions later on.

We also filter the dataset to include only entries up to (and including) the year 2024, since 2025 is incomplete at the time of analysis.

```sql
create table HousePricesRegional as 
select
    -- convert text to datetime using STR_TO_DATE, then extract the date only using DATE
	DATE(STR_TO_DATE(Date, '%d/%m/%Y %H:%i')) as date,    
	CAST(regionname AS CHAR(50)) as region,
    AveragePrice as average_price,
    -- for the remaining columns, we convert the text to a double (by using the value in a numerical context). 
    -- If there is no value, we assign NULL to it.
    case
		when salesvolume != '' then salesvolume + 0
		else null
	end as sales_volume,
	case
		when DetachedPrice != '' then DetachedPrice + 0
		else null
	end as detached_avg,  
	case
		when SemiDetachedPrice != '' then SemiDetachedPrice + 0
		else null
	end as semi_avg,  
	case
		when TerracedPrice != '' then TerracedPrice + 0
		else null
	end as terraced_avg, 
	case
		when FlatPrice != '' then FlatPrice + 0
		else null
	end as flat_avg
from hpiinitialtransform
where YEAR(DATE(STR_TO_DATE(Date, '%d/%m/%Y %H:%i'))) <= 2024; -- remove 2025 data
```

To ensure intregity of the data, we add foreign key constraints to the `date` and `region` columns, referencing their respective dimension tables. If this works without errors, we know that the `date` and `region` values in the new table are of the required form. 

![foreign key constraints worked with no errors](documentation_images\FK-Housing-Works.png)

We are now done with the original table, `hpiinitialtranform`, so we drop it to keep our schema tidy.

```sql
drop table hpiinitialtransform
```

The query(ies) for creating the national house prices table is almost identical (in fact, the structure is the exact same once all statements involving regions are removed). 

For completeness, we supply the queries, but do not comment on them further. The new table is called `HousePricesNational`.

```sql
create table HousePricesNational as 
select
    -- convert text to datetime using STR_TO_DATE, then extract the date only using DATE
	DATE(STR_TO_DATE(Date, '%d/%m/%Y %H:%i')) as date, 
    AveragePrice as average_price,
    -- for the remaining columns, we convert the text to a double (by using the value in a numerical context). 
    -- If there is no value, we assign NULL to it.
    case
		when salesvolume != '' then salesvolume + 0
		else null
	end as sales_volume,
	case
		when DetachedPrice != '' then DetachedPrice + 0
		else null
	end as detached_avg,  
	case
		when SemiDetachedPrice != '' then SemiDetachedPrice + 0
		else null
	end as semi_avg,  
	case
		when TerracedPrice != '' then TerracedPrice + 0
		else null
	end as terraced_avg, 
	case
		when FlatPrice != '' then FlatPrice + 0
		else null
	end as flat_avg
from hpinational
where YEAR(DATE(STR_TO_DATE(Date, '%d/%m/%Y %H:%i'))) <= 2024; -- remove 2025 data

alter table HousePricesNational
ADD FOREIGN KEY (date) REFERENCES DatesDim(date);

drop table hpinational;
```

## Median Salary tables (regional walkthrough, national is essentially identical)

We query for a description as well as a random sample of 10 rows to see how the data from the `medianpayinitialtransform` looks.

```sql
desc medianpayinitialtransform;

select * from medianpayinitialtransform order by rand();
```

Outputs:

![the output of the description and random sample of the median pay regional table](documentation_images\MedianSalaryFirstLook.png)


We see that some region names have whitespace before them, this is verified by the following query, so we will have to correct this when cleaning the data (using the `TRIM` function):

```sql
select region from medianpayinitialtransform where region = '        East';
```

![output of the whtiespace issue](documentation_images\WhiteSpaceIssue.png)

Otherwise, things look good, so we proceed to create the new table, which we will call `MedianSalaryRegional`. This includes creating a reference to the `RegionsDim` table.

```sql
create table MedianSalaryRegional as 
select
    -- convert text to datetime using STR_TO_DATE, then extract the date only using DATE
	cast(TRIM(region) as char(30)) as region, 
    year as year,
    mediansalary as salary
from medianpayinitialtransform
where year <= 2024; -- remove any 2025 data, if it exists (it doesn't, but just to be safe)
```

However, when trying to add a foreign key constraint via

```sql 
alter table MedianSalaryRegional
add foreign key (region) references RegionsDim(region);
```

we receive a 1452 error, stating that "a foreign key constraint fails...". We can investigate this by querying for the regions which are in the `MedianSalaryRegional` table, but not in the `RegionsDim` table, using the query

```sql
select distinct region from MedianSalaryRegional where region not in (select region from regionsdim);
```

This query returns "East". Some investigation finds that the issue is that in the RegionsDim table, "East" is referred to as "East of England". 

We therefore replace "East" with "East of England" in the `regions` column of `MedianSalaryRegional` and reattempt the foreign key constraint.

```sql 
update MedianSalaryRegional
set region = "East of England"
where region = "East";

alter table MedianSalaryRegional
add foreign key (region) references RegionsDim(region);
```

It works! We finish with the regional median salary data by dropping the original table:

```sql
drop table medianpayinitialtransform;
```


For the national table, there is not much to do. The table `mediansalarynational` is already in the form we want. For example, the columns are of the correct type, as can be seen from querying for a description, and also the maximum year is 2023, which is within the range we want. Therefore, we leave it as it is.

```sql
desc MedianSalaryNational;

-- check that the maximum year isn't 2025 (as we don't want partial 2025 data)
select max(year) from MedianSalaryNational;
```

Outputs:

![output of querying description and maximum year of the MedianSalaryNational table](documentation_images\MedianSalaryNationalOK.png)


## CPIH table

We query for a description and a random sample of 10 rows in order to see what the data looks like.

```sql
desc cpihinitialtransform;

select * from cpihinitialtransform order by rand();
```

Outputs:

![the output of querying for a description and random sample from the cpi table](documentation_images\CPIFirstLook.png)

We can see that `MonthYear` is of type "Text", and the format is an abbreviated month name and then a 4-digit year separated by a space. This is given as "%b %Y" as a format string. We use this alongside the `STR_TO_DATE` function to convert the column to datetimes, then we use `MONTH` and `YEAR` to create month and year columns. 

We make use of a CTE within the `create table` statement so that we only have to convert the text to dates once.

We also add a column normalising the CPI indices so that Dec 2024's index is 1 (and so most normalised historical indices will be <1 - this will be useful when adjusting housing prices by CPIH later).

```sql
create table CPIH as 
	with formatted_date as
		(
		select DATE(str_to_date(concat(MonthYear,' 01'), "%Y %b %d")) as f_date, `CPI-Index` from cpihinitialtransform
		)
	select
		MONTH(f_date) as month, 
		YEAR(f_date) as year, 
		`CPI-Index` as indx, -- to avoid error with using the index keyword (i.e. indx instead of index).
        -- normalise by the most recent CPI-Index, unless it is more recent than 2024-12-01. If so, normalise by CPI-Index from 2024-12-01 instead.
		`CPI-Index`/(select `CPI-Index` 
                    from formatted_date where f_date = (select least(max(f_date),"2024-12-01") from formatted_date)
                    ) as index_normed
	from formatted_date
	where YEAR(f_date) <= 2024; -- remove 2025 data if it exists
    ```

A sample from the newly created CPIH table shows that the creation has worked as intended. We can drop the old CPI table, namely `cpiinitialtransform`, from our schema to keep it tidy.

```sql
select * from CPIH limit 10;
    
drop table cpihinitialtransform;
```

![a sample of the new CPI with the dates in a more workable format](documentation_images\NewCPISample.png)

# Conclusion

The modelling phase is now finished. Our final schema is as follows:

![image of the final schema](documentation_images\FinalSchema.png)

Next, we’ll focus on generating insights using SQL and then move on to creating a report in Power BI. In Power BI, we’ll likely need to create a separate dimension table for `Year` to aid in filtering and to ensure the correct context for visuals.